In [11]:
import pandas as pd
import matplotlib as mp
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import xgboost as xgb

%matplotlib inline

#Root de archivos de datos
root_dir = '../data/'
group=['ref_hash','window_nr']

---
## Read de Features

In [12]:
features = pd.read_csv('training_set.csv').set_index(group)

In [13]:
features.head()

,,secs_to_next_mean,secs_since_last_arrival,auctions_total,auctions_last_hour,amount_auctions_in_weekend,is_last_weekend,amount_dif_src,amount_events,secs_since_last_event,wifi,timeToClick_mean,amount_dif_advertisers
ref_hash,window_nr,,,,,,,,,,,,
2564673204772915246,1,765.0,88625.0,216.0,0.0,0.0,False,3.0,0.0,0.0,False,197.830534,0.0
4441121667607578179,1,111.0,29052.0,1188.0,0.0,640.0,True,5.0,0.0,0.0,False,197.830534,0.0
7721769811471055264,1,281.0,333.0,908.0,23.0,218.0,True,3.0,99.0,196475.0,False,197.830534,0.0
1258642015983312729,1,8365.0,121224.0,12.0,0.0,0.0,False,1.0,0.0,0.0,False,197.830534,0.0
6707090658317158573,1,453.0,4.0,516.0,35.0,171.0,True,4.0,0.0,0.0,False,197.830534,0.0


In [14]:
targets = pd.read_csv('targets.csv').set_index(group)

In [15]:
targets.isnull().sum()

target_st    0
target_sc    0
dtype: int64

In [16]:
training_set = features.merge(targets, how='inner', left_index=True,right_index=True)

In [17]:
training_set.head()

,,secs_to_next_mean,secs_since_last_arrival,auctions_total,auctions_last_hour,amount_auctions_in_weekend,is_last_weekend,amount_dif_src,amount_events,secs_since_last_event,wifi,timeToClick_mean,amount_dif_advertisers,target_st,target_sc
ref_hash,window_nr,,,,,,,,,,,,,,
4881735474151208235,1,8032.629065,0.0,0.0,0.0,0.0,False,0.0,2.0,23418.0,False,197.830534,0.0,259200.0,259200.0
4679881814211948892,1,45.000000,156639.0,5.0,0.0,0.0,False,2.0,0.0,0.0,False,25.335000,1.0,63602.0,259200.0
4177895333092974918,1,3990.000000,119428.0,15.0,0.0,0.0,False,2.0,0.0,0.0,False,197.830534,0.0,259200.0,259200.0
4312326900897087220,1,4703.000000,10305.0,53.0,0.0,22.0,True,4.0,0.0,0.0,False,197.830534,0.0,8056.0,259200.0
4911541169216784265,2,7893.726152,0.0,0.0,0.0,0.0,False,0.0,14.0,69048.0,False,186.101670,0.0,259200.0,259200.0


In [18]:
x_st = training_set[[x for x in training_set.columns if x not in targets.columns]]
y_st = training_set['target_st']

x_sc = training_set[[x for x in training_set.columns if x not in targets.columns]]
y_sc = training_set['target_sc']

---
## Training 

In [19]:
xtrain_st, xtest_st, ytrain_st, ytest_st = train_test_split(x_st, y_sc, train_size=0.7)
xtrain_sc, xtest_sc, ytrain_sc, ytest_sc = train_test_split(x_sc, y_sc, train_size=0.7)

In [20]:
model_st = RandomForestRegressor(random_state=0, n_estimators=20, n_jobs=-1)
model_sc = RandomForestRegressor(random_state=0, n_estimators=20, n_jobs=-1)

model_st.fit(xtrain_st, ytrain_st)
model_sc.fit(xtrain_sc, ytrain_sc)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=-1,
                      oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [21]:
# 1. Solo Features Nico: 
#    Error St.: 3568912651.823991
#    Error Sc.: 3593870054.4797
    
ypred_st = model_st.predict(xtest_st)
print("Error St.:", mean_squared_error(ypred_st,ytest_st))

ypred_sc = model_sc.predict(xtest_sc)
print("Error Sc.:", mean_squared_error(ypred_sc,ytest_sc))

Error St.: 2881492692.2741113
Error Sc.: 2875116068.968186


In [22]:
model_st.feature_importances_

array([0.23180716, 0.2492613 , 0.10554769, 0.01201559, 0.07115562,
       0.00531138, 0.04751844, 0.03711925, 0.2151725 , 0.00615583,
       0.01726011, 0.00167514])

In [23]:
model_sc.feature_importances_

array([0.230021  , 0.24960007, 0.10581098, 0.01170499, 0.07205876,
       0.00491372, 0.04788891, 0.03361869, 0.21943009, 0.00694882,
       0.01618829, 0.00181567])

---
## Prediccion

In [24]:
# Labels a submitir con las predicciones

to_predict = pd.read_csv(root_dir + 'target_competencia_ids.csv',usecols=['ref_hash'])
to_predict.head()

,ref_hash
0,1000169251625791246_sc
1,1000169251625791246_st
2,1000395625957344683_sc
3,1000395625957344683_st
4,1003027494996471685_sc


In [25]:
X = features.reset_index()
X = X.loc[X['window_nr'] == 3].set_index(group)

In [26]:
y_st = model_st.predict(X)
y_sc = model_sc.predict(X)

In [27]:
results_st = X.reset_index()['ref_hash'].transform(lambda x: str(x) + '_st').rename('ref_hash').to_frame()
results_st['obj'] = y_st

results_sc = X.reset_index()['ref_hash'].transform(lambda x: str(x) + '_sc').rename('ref_hash').to_frame()
results_sc['obj'] = y_sc

results = results_st.append(results_sc)

In [28]:
#Armado de File Final
max_secs = 3*24*60*60
to_predict.merge(results, on='ref_hash', how='left').fillna(max_secs).to_csv('predictions_rf.csv', header=True, index=False)

In [29]:
to_predict.count()

ref_hash    8074
dtype: int64